# 自定义一些结构
- 激活函数
- nn module
- 动态网络、权重共享网络

## 1. 自定义激活函数等网络结构
定义为 torch.autograd.Function 的子类，并实现 forward 和 backward函数。

In [4]:
import torch

class MyReLU(torch.autograd.Function): 
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数， 并完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x): 
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量; 我们必须返回一个包含输出的张量， 并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。 
        """
        ctx.save_for_backward(x) 
        return x.clamp(min=0)
    @staticmethod
    def backward(ctx, grad_output): 
        """
        在反向传播中，我们接收到上下文对象和一个张量，其包含了相对于正向传播过程中产生的输出的损失的梯度。 我们可以从上下文对象中检索缓存的数据， 并且必须计算并返回与正向传播的输入相关的损失的梯度。 
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone() 
        grad_x[x < 0] = 0
        return grad_x

In [ ]:
# 用的时候
y_pred = MyReLU.apply(x)

## 2. 自定义nn module
nn相当于Keras里封装的层

In [8]:
import torch
# N是批大小;D是输入维度
# H是隐藏层维度;D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
#创建输入和输出随机张量
x = torch.randn(N, D_in) 
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。 # 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。 # 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(torch.nn.Linear(D_in, H), torch.nn.ReLU(), torch.nn.Linear(H, D_out))

In [9]:
# 自定义nn module
import torch
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """ 在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。 """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
    def forward(self, x): 
        """在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。 我们可以使用构造函数中定义的模块以及张量上的任意的(可微分的)操作。 """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu) 
        return y_pred
# N是批大小; D_in 是输入维度;
# H 是隐藏层维度; D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
# 产生输入和输出的随机张量
x = torch.randn(N, D_in) 
y = torch.randn(N, D_out)
# 通过实例化上面定义的类来构建我们的模型。 model = TwoLayerNet(D_in, H, D_out)

## 3. 动态图、权重共享
pytorch动态图和tensorflow静态图的区别：
- 动态图：每一个前向通道定义一个新的计算图。好处是可以通过命令控制流，也很方便实现权重共享。
- 静态图：定义计算图一次，然后重复执行这个相同的图，可能会提供不同的输入数据。

In [12]:
import random 
import torch
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """ 在构造函数中，我们构造了三个nn.Linear实例，它们将在前向传播时被使用。 """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H) 
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
    def forward(self, x): 
        """
        对于模型的前向传播，我们随机选择0、1、2、3， 并重用了多次计算隐藏层的middle_linear模块。 
        由于每个前向传播构建一个动态计算图， 我们可以在定义模型的前向传播时使用常规Python控制流运算符，如循环或条件语句。 
        在这里，我们还看到，在定义计算图时多次重用同一个模块是完全安全的。
        这是Lua Torch的一大改进，因为Lua Torch中每个模块只能使用一次。 """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)): #用命令可控制流，每次前向传播的计算图可不一样
            h_relu = self.middle_linear(h_relu).clamp(min=0) # 通过重用同一个模块，实现参数共享
        y_pred = self.output_linear(h_relu)
        return y_pred

# N是批大小;D是输入维度
# H是隐藏层维度;D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10
# 产生输入和输出随机张量
x = torch.randn(N, D_in) 
y = torch.randn(N, D_out)
# 实例化上面定义的类来构造我们的模型 
model = DynamicNet(D_in, H, D_out)
# 构造我们的损失函数(loss function)和优化器(Optimizer)。
# 用平凡的随机梯度下降训练这个奇怪的模型是困难的，所以我们使用了momentum方法。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9) 
for t in range(500):
    # 前向传播:通过向模型传入x计算预测的y。 
    y_pred = model(x)
    # 计算并打印损失
    loss = criterion(y_pred, y) 
    print(t, loss.item())
    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step()

0 645.8611450195312
1 646.2496337890625
2 643.92626953125
3 642.2386474609375
4 679.82861328125
5 623.885009765625
6 536.7086181640625
7 630.441650390625
8 627.3451538085938
9 623.6453247070312
10 619.084228515625
11 648.8336791992188
12 631.8842163085938
13 631.0870971679688
14 629.8173217773438
15 243.30859375
16 219.35560607910156
17 610.2099609375
18 146.75283813476562
19 582.1411743164062
20 575.20068359375
21 82.82259368896484
22 74.67125701904297
23 607.9439086914062
24 602.2174072265625
25 522.7252197265625
26 482.0063781738281
27 566.7910766601562
28 419.6882629394531
29 430.8864440917969
30 398.78363037109375
31 364.791015625
32 162.037353515625
33 409.680908203125
34 268.6021423339844
35 248.32708740234375
36 215.49024963378906
37 139.56686401367188
38 155.86228942871094
39 207.582763671875
40 90.24348449707031
41 73.28643798828125
42 54.25720977783203
43 111.40339660644531
44 33.15079879760742
45 30.527606964111328
46 28.668384552001953
47 26.151016235351562
48 23.316341400

394 1.226511001586914
395 0.25454431772232056
396 0.23428285121917725
397 0.7104739546775818
398 0.08954990655183792
399 0.4096643328666687
400 0.1717688888311386
401 0.919624388217926
402 0.7599819898605347
403 0.39124932885169983
404 0.6929718852043152
405 0.2672030031681061
406 0.16806915402412415
407 0.10168235003948212
408 1.8093149662017822
409 0.0950983464717865
410 0.4012102782726288
411 0.6039052605628967
412 0.07102047652006149
413 0.059217408299446106
414 0.4165453016757965
415 0.05834047496318817
416 0.38559457659721375
417 0.09446528553962708
418 0.28021639585494995
419 0.21727773547172546
420 2.17871356010437
421 0.8592859506607056
422 1.4793391227722168
423 1.9032708406448364
424 0.7935872673988342
425 0.6791202425956726
426 0.9484685659408569
427 0.23213127255439758
428 1.1350268125534058
429 0.5613948106765747
430 0.1268453449010849
431 0.7313069105148315
432 0.44078901410102844
433 0.1431128978729248
434 1.15078866481781
435 0.1358259618282318
436 1.0001493692398071
4